In [ ]:
!pip install evaluate


In [ ]:
!pip install transformers datasets accelerate peft bitsandbytes


In [ ]:
from datasets import load_dataset

dataset = load_dataset("ShenLab/MentalChat16K")

In [ ]:
print(dataset)
print(dataset['train'][0])


DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 16084
    })
})
{'instruction': "You are a helpful mental health counselling assistant, please answer the mental health questions based on the patient's description. \nThe assistant gives helpful, comprehensive, and appropriate answers to the user's questions. ", 'input': "I've been struggling with my mental health for a while now, and I can't seem to find a way to cope with it. I've tried visualization, positive thinking, and even medication, but nothing seems to work. I've been feeling lost and helpless, and I don't know what to do next. My mind is a whirlwind of thoughts and emotions, and I can't seem to make sense of it all. I feel like I'm drowning in a sea of confusion, and I can't seem to find my way out.", 'output': "I understand that you've been dealing with a sense of confusion and chaos in your thoughts and emotions for some time now. It's been a challenging journey, an

In [ ]:
# Install required packages
!pip install google-generativeai datasets

# Import Libraries
import google.generativeai as genai
from datasets import load_dataset

# Set your API key (replace with your key, but NEVER hard-code it in production)
api_key = "YOUR_API_KEY"  # <- Replace this temporarily in your local code

# Configure Gemini
genai.configure(api_key=api_key)
model = genai.GenerativeModel('gemini-1.5-pro')

# Load MentalChat16K dataset
dataset = load_dataset("ShenLab/MentalChat16K")


In [ ]:
def generate_prompt(user_input, dataset):
    # Loop through dataset safely
    for example in dataset['train']:
        input_text = example.get('input')
        response_text = example.get('output')

        # Skip if input or response is missing
        if input_text is None or response_text is None:
            continue


        if user_input.lower() in input_text.lower():
            return f"""You're a helpful and empathetic mental health assistant.

Example conversation:
User: {input_text}
Assistant: {response_text}

Now help this user:
User: {user_input}
Assistant:"""

    # Fallback prompt if no similar example is found
    return f"""You're a helpful and empathetic mental health assistant.

User: {user_input}
Assistant:"""

#  chat with Gemini
def chat_with_gemini(user_input, dataset):
    prompt = generate_prompt(user_input, dataset)


    response = model.generate_content(prompt)


    return response.text

#   example
user_input = "I feel anxious"
response = chat_with_gemini(user_input, dataset)
print("Gemini:", response)


Gemini: I hear you. Feeling anxious can be really uncomfortable and unsettling. Can you tell me a little more about what's making you feel this way?  Sometimes just talking about it can help.  It's also okay if you don't feel comfortable sharing the specifics right now.  We can still explore some ways to help you manage the anxiety.  Would you be open to trying some relaxation techniques like deep breathing or mindfulness exercises? Or perhaps we could talk about some coping strategies you've used in the past that have been helpful. I'm here to listen and support you in whatever way feels best.



In [4]:
!pip install rouge_score
!pip install evaluate

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=1be097eac309aa0ef0097858eb01645d2f52df236ff96f2ff090729407334876
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's depen

In [8]:
import json
import google.generativeai as genai
from tqdm import tqdm
import evaluate
import pandas as pd

# رفع الملف هنا
from google.colab import files
uploaded = files.upload()

# اسم الملف الذي تم رفعه
file_name = list(uploaded.keys())[0]

# قراءة البيانات من الملف
with open(file_name, 'r') as f:
    data = json.load(f)

# Configure Gemini API
api_key = "Your API KEY"
genai.configure(api_key=api_key)
model = genai.GenerativeModel("gemini-1.5-pro")

# Evaluation Metrics
bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")

# Define improved prompt styles (hyperparameter: prompt_template)
prompt_templates = [
    lambda q: f"You are a compassionate assistant. Help the user with their mental health concern by providing a calming and empathetic response.\nUser: {q}\nAssistant:",
    lambda q: f"As a therapist, respond empathetically to this user message by offering helpful advice:\n{q}\nResponse:",
    lambda q: f"""You're a supportive mental health chatbot. The user says: "{q}". Provide a thoughtful and empathetic response to support them emotionally."""
]

# Store results
results = []

# Loop through prompt styles (hyperparameter tuning)
for i, prompt_func in enumerate(prompt_templates):
    print(f"Evaluating with Prompt Style {i+1}...")

    predictions = []
    references = []

    for sample in tqdm(data):
        question = sample["question"]
        expected_answer = sample["answer"]

        prompt = prompt_func(question)

        try:
            response = model.generate_content(prompt)
            prediction = response.text.strip()
        except Exception as e:
            prediction = ""

        # Ensure prediction is non-empty
        if prediction:
            predictions.append(prediction)
            references.append([expected_answer])  # Ensure that each reference is a list
        else:
            print(f"Empty prediction for question: {question}")

    # Format the predictions and references
    formatted_predictions = [' '.join(p.split()) for p in predictions]  # Join words if necessary
    formatted_references = [[' '.join(r[0].split())] for r in references]  # Ensure that references are lists of sentences

    # Ensure no division by zero in BLEU calculation
    if len(formatted_predictions) > 0 and len(formatted_references) > 0:
        bleu_score = bleu_metric.compute(predictions=formatted_predictions, references=formatted_references)
    else:
        bleu_score = {"bleu": 0.0}  # Return BLEU score of 0.0 if no valid predictions/references

    # Compute ROUGE
    rouge_score = rouge_metric.compute(predictions=formatted_predictions, references=formatted_references,
                                       rouge_types=["rouge1", "rouge2", "rougeL"])

    # Store results
    results.append({
        "prompt_style": i + 1,
        "BLEU": bleu_score["bleu"],
        "ROUGE-1": rouge_score["rouge1"],  # Access the float value directly
        "ROUGE-2": rouge_score["rouge2"],  # Access the float value directly
        "ROUGE-L": rouge_score["rougeL"],  # Access the float value directly
    })

# Display final scores
df = pd.DataFrame(results)
print(df)


Saving colab_gpu_limited_100 1.json to colab_gpu_limited_100 1 (3).json
Evaluating with Prompt Style 1...


  1%|          | 1/100 [00:00<01:05,  1.50it/s]

Empty prediction for question: I hadn't thought about that. I've been so focused on the physical aspect of her illness that I've neglected the mental and emotional aspects. I remember how effective therapy was for her when she was dealing with depression years ago. I'll reach out to her healthcare team and discuss alternative approaches to help her regain her appetite and improve her overall condition.


  2%|▏         | 2/100 [00:01<01:16,  1.28it/s]

Empty prediction for question: I've always been a diligent patient, relying on my doctor's expertise for all my health concerns. But lately, I've found myself drawn to the internet when I'm unsure about my symptoms. The problem is, the information I find online is often conflicting and unreliable, leaving me more confused than before.


...

Empty prediction for question: You're right. I've been so focused on the debt that I've forgotten about the progress I've made. I've been saving up for a rainy day, and I've even started to pay off some of my smaller debts.


 99%|█████████▉| 99/100 [03:05<00:00,  1.55it/s]

Empty prediction for question: I've been feeling helpless and hopeless since my husband fell ill. The thought of leaving him behind in the [Country], where we can't afford proper medical care, tears me apart. Our financial situation has only worsened since his illness, and I'm struggling to make ends meet.


100%|██████████| 100/100 [03:06<00:00,  1.86s/it]


Empty prediction for question: Lately, it seems that every small setback triggers an overwhelming sense of despair within me. Negative thoughts consume my mind, telling me that I am worthless and unlovable. The continuous cycle of self-criticism has led to decreased motivation and interest in things I used to enjoy. My goal is to address these feelings of worthlessness and develop healthier thought patterns through therapy.
   prompt_style      BLEU   ROUGE-1   ROUGE-2   ROUGE-L
0             1  0.052677  0.387551  0.108039  0.203652
1             2  0.047958  0.385426  0.096169  0.175744
2             3  0.051229  0.380669  0.104934  0.179809
